<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/TwelveLabs_Quickstart_Search.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Search
This guide shows how to utilize the TwelveLabs Python SDK for searching within your video content.

# Prerequisites
Before you begin, ensure the following prerequisites are met:

- [Sign up](https://playground.twelvelabs.io/) for a free account and obtain your API key from the [API Key](https://playground.twelvelabs.io/dashboard/api-key) page. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project. 
- The videos you wish to upload must meet the requirements in the [Prerequisites](https://docs.twelvelabs.io/docs/upload-single-videos#prerequisites) section of the **Upload single videos page**.



# Procedure

## Install the TwelveLabs Python SDK

In [ ]:
%pip install -U -q twelvelabs

## Import the required packages

In [5]:
import os
from glob import glob
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task

## Configure your API key


In [ ]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get('TL_API_KEY')

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## Create an index

An index is a basic unit for organizing and storing video data consisting of video embeddings and metadata. Indexes facilitate information retrieval and processing.

When creating a new index, you must specify at least the following information:
- **Name**: Use a brief and descriptive name to facilitate future reference and management.
- **Model configuration**: Provide a list containing the video understanding models and the associated model options you want to enable.

TwelveLabs provides two distinct model types:

- **Embedding models (Marengo)**: This model performs tasks such as search and embedding generation , enabling enhanced video understanding.
- **Generative models (Pegasus)**: This model generates text based on your videos.

Model options determine what information the video understanding model will process:
- **visual**: Analyzes visual content, including actions, objects, events, text (OCR), and brand logos.
- **audio**: Analyzes audio content, including ambient sounds and human speech.

In [12]:
client = TwelveLabs(api_key=TL_API_KEY)

index = client.index.create(
    name="Temp-1",
    models=[
        {
            "name": "marengo2.7",
            "options": ["visual", "audio"],
        }
    ]
)
print(f"Created index: id={index.id} name={index.name} models={index.models}")


Created index: id=67c836599d39a8522d5950e6 name=Temp-1 models=root=[Model(name='marengo2.7', options=['visual', 'audio'], addons=None, finetuned=False)]


## Upload videos to your index

In [13]:
# The path to the directory containing the videos you wish to upload.
VIDEO_PATH = "funny-cats.mp4"  # In a colab Notebook, just upload the file in the root folder and just give the filename.

video_files = glob(VIDEO_PATH)
if(len(video_files)==0):
  raise Exception("Video file was not provided")
for video_file in video_files:
  print(f"Uploading {video_file}")
  task = client.task.create(index_id=index.id, file=video_file)
  print(f"Created task: id={task.id}")

  # (Optional) Monitor the video indexing process
  # Utility function to print the status of a video indexing task
  def on_task_update(task: Task):
          print(f"  Status={task.status}")
  task.wait_for_done(sleep_interval=50, callback=on_task_update)
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_file}. The unique identifer of your video is {task.video_id}.")


Uploading funny-cats.mp4
Created task: id=67c836dbd4e0d626915c0d05
  Status=indexing
  Status=ready
Uploaded funny-cats.mp4. The unique identifer of your video is 67c836dbaee89d69a9c6a096.


## Perform searches

**Using text queries**:

In [19]:
# Search Query
search_results = client.search.query(index_id=index.id, query_text="Cats doing funny things", options=["visual", "audio"])

# Utility function to print a specific page
def print_page(page):
  for clip in page:
    print(
        f" video_id={clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence}"
    )
print_page(search_results.data)
while True:
    try:
        print_page(next(search_results))
    except StopIteration:
        break

 video_id=67c836dbaee89d69a9c6a096 score=84.76 start=562.533327708 end=575.633327577 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.7 start=15.133333182 end=37.466666292 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.67 start=541.33332792 end=547.66666119 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.66 start=218.499997815 end=232.933331004 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.66 start=380.783329525 end=407.499995925 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.64 start=181.499998185 end=197.399998026 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.64 start=441.799995582 end=447.066662196 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.6 start=53.66666613 end=59.233332741 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.59 start=75.466665912 end=90.366665763 confidence=high
 video_id=67c836dbaee89d69a9c6a096 score=84.59 start=339.433329939 end=362.933329704 confidence=high
 

**Using image queries**

In [ ]:
search_results = client.search.query(
    index_id=index.id,
    query_media_type="image",
    query_media_file="<YOUR_FILE_PATH>", # Use query_media_url instead to provide a file from a publicly accessible URL.
    options=["visual"]
)

# Utility function to print a specific page
def print_page(page):
  for clip in page:
    print(
        f" video_id={clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence}"
    )

print_page(search_results.data)

while True:
    try:
        print_page(next(search_results))
    except StopIteration:
        break